In [35]:
# Day 8 Part 1

def get_accumulator_value(instructions):
    idx = 0
    accumulator = 0
    instructions = [[x[:3], int(x[4:]), False] for x in instructions]
    
    while(True):
        if instructions[idx][2]:
            return accumulator
        
        instructions[idx][2] = True
        if instructions[idx][0] == 'nop':
            idx = idx + 1
        elif instructions[idx][0] == 'acc':
            accumulator = accumulator + instructions[idx][1]
            idx = idx + 1
        elif instructions[idx][0] == 'jmp':
            idx = idx + instructions[idx][1]
    
# Test first
with open('test_data/day8.txt', 'r') as in_file:
    test_data = in_file.read().split('\n')

print("Running on test data. Verifying result...")
print(get_accumulator_value(test_data) == 5)
print('')

# Now run on actual data
with open('data/day8.txt', 'r') as in_file:
    real_data = in_file.read().split('\n')

print("Puzzle answer is: " + str(get_accumulator_value(real_data)))

Running on test data. Verifying result...
True

Puzzle answer is: 1675


In [46]:
# Day 8 Part 2

import copy

def get_accumulator_value2(instructions):
    idx = 0
    accumulator = 0
    
    while(True):                
        if idx == len(instructions):
            return True, accumulator
        if instructions[idx][2]:
            return False, accumulator
        
        instructions[idx][2] = True
        if instructions[idx][0] == 'nop':
            idx = idx + 1
        elif instructions[idx][0] == 'acc':
            accumulator = accumulator + instructions[idx][1]
            idx = idx + 1
        elif instructions[idx][0] == 'jmp':
            idx = idx + instructions[idx][1]

def fix_infinite_loop(instructions):
    instructions = [[x[:3], int(x[4:]), False] for x in instructions]
    for i in range(0, len(instructions)):
        inst_copy = copy.deepcopy(instructions)
        if inst_copy[i][0] == 'nop':
            inst_copy[i][0] = 'jmp'
        elif inst_copy[i][0] == 'jmp':
            inst_copy[i][0] = 'nop'
        else:
            continue
        
        success, acc = get_accumulator_value2(inst_copy)
        if success:
            return success, acc
    return False, -1
            
    
print("Running on test data. Verifying result...")
print(fix_infinite_loop(test_data) == (True, 8))
print('')

# Now run on actual data
print("Puzzle answer is: " + str(fix_infinite_loop(real_data)[1]))

Running on test data. Verifying result...
True

Puzzle answer is: 1532


In [63]:
# Day 9 Part 1

import numpy as np

def can_find_two_entries(data, target_number):
    for item in data:
        temp = target_number - item
        try:
            data.tolist().index(temp)
            return True
        except:
            pass
    return False

def find_weakness(data, preamble_size):
    for i in range(preamble_size, len(data)):
        preamble = np.sort(data[i-preamble_size:i])
        number = data[i]
        if not can_find_two_entries(preamble, number):
            return number

test_data = np.loadtxt('test_data/day9.txt', dtype=np.int64)
print("Running on test data. Verifying result...")
print(find_weakness(test_data, 5) == 127)
print('')

real_data = np.loadtxt('data/day9.txt', dtype=np.int64)
print("Puzzle answer is: " + str(find_weakness(real_data, 25)))

Running on test data. Verifying result...
True

Puzzle answer is: 1309761972


In [68]:
# Day 9 Part 2

def find_contiguous_sum(data, list_width, target_number):
    for i in range(0, len(data) - list_width):
        if np.sum(data[i:i + list_width]) == target_number:
            return data[i:i + list_width]
    
def crack_encryption(data, target_number):
    for list_width in range(2, len(data)-1):
        numbers = find_contiguous_sum(data, list_width, target_number)
        if numbers is not None:
            numbers = np.sort(numbers)
            return numbers[0] + numbers[list_width-1]
    
test_data = np.loadtxt('test_data/day9.txt', dtype=np.int64)
print("Running on test data. Verifying result...")
print(crack_encryption(test_data, 127) == 62)
print('')

real_data = np.loadtxt('data/day9.txt', dtype=np.int64)
print("Puzzle answer is: " + str(crack_encryption(real_data, 1309761972)))

Running on test data. Verifying result...
True

Puzzle answer is: 177989832


In [115]:
# Day 10 Part 1

def count_1_and_3_jolt_differences(data):
    data = np.sort(data)
    diff1 = 0
    diff3 = 1  #always will have a diff3 at the end
    if data[0] == 1:
        diff1 = 1
    if data[0] == 3:
        diff3 = diff3 + 1
    
    for i in range(1, len(data)):
        if data[i] - data[i-1] == 1:
            diff1 = diff1 + 1
        if data[i] - data[i-1] == 3:
            diff3 = diff3 + 1
    return diff1, diff3
            
test_data = np.loadtxt('test_data/day10.txt', dtype=np.int32)
test_data2 = np.loadtxt('test_data/day10_larger.txt', dtype=np.int32)
print("Running on test data. Verifying result...")
print(count_1_and_3_jolt_differences(test_data) == (7, 5))
print(count_1_and_3_jolt_differences(test_data2) == (22, 10))
print('')

real_data = np.loadtxt('data/day10.txt', dtype=np.int32)
diff1, diff3 = count_1_and_3_jolt_differences(real_data)
print("Puzzle answer is: " + str(diff1*diff3))

Running on test data. Verifying result...
True
True

Puzzle answer is: 2590


In [130]:
# Day 10 Part 2
from math import comb

count_cache = {}

def prepare_data(data):
    sorted_data = np.sort(data)
    temp = [0]
    temp.extend([x for x in sorted_data])
    temp.append(sorted_data[-1] + 3)
    return temp

def dynamic_count_combinations(data, i):
    if i == len(data) - 1:
        return 1
    
    if i in count_cache:
        #print(count_cache)
        return count_cache[i]
    
    count = 0
    for j in range(i + 1, len(data)):
        if data[j] - data[i] <= 3:
            count += dynamic_count_combinations(data, j)

    count_cache[i] = count
    return count

def count_combinations(data):
    count_cache.clear()
    return dynamic_count_combinations(prepare_data(data), 0)

print("Running on test data. Verifying result...")
print(count_combinations(test_data) == 8)
print(count_combinations(test_data2) == 19208)
print('')

print("Puzzle answer is: " + str(count_combinations(real_data)))

Running on test data. Verifying result...
True
True

Puzzle answer is: 226775649501184
